## 依赖包
<!-- **Author**: `zwy` -->


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

import random
import shutil

plt.ion()   # 交互模式可以动态显示图像

### 硬件选择 GPU

In [2]:
# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 数据加载及处理


### 下载数据集
可以从Google Drive / Github / 其他数据地址下载数据

In [ ]:
!rm -rf CNN
!git clone https://github.com/Zwysun/CNN.git
!ls

Cloning into 'CNN'...
remote: Enumerating objects: 705, done.
remote: Counting objects: 100% (705/705), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 705 (delta 163), reused 680 (delta 141), pack-reused 0
Receiving objects: 100% (705/705), 31.29 MiB | 15.97 MiB/s, done.
Resolving deltas: 100% (163/163), done.
CNN  sample_data


### 数据预处理
利用transforms对图片进行预处理，可以分别针对训练集和验证集采取不同的处理方法


In [3]:
# 直接加载数据集CIFAR10
# 正则化

# 图像变换操作
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])
    ]),
    'test': transforms.Compose([
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        # transforms.Normalize([0.5], [0.5])
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261])
    ]),
}

# 下载公共数据集

train_dataset = datasets.CIFAR10(root="./cifar10",  # 设置数据集的根目录
    train=True,  # 是否是训练集
    transform=data_transforms['train'],  # 对数据进行转换
    download=True
    )
test_dataset = datasets.CIFAR10(root="./cifar10", 
    train=False,  # 测试集，所以false
    transform=data_transforms['test'], 
    download=True
    )


Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified


In [4]:
image_datasets = {'train':train_dataset, 'test':test_dataset}
# 数据
dataloaders = {x:torch.utils.data.DataLoader(image_datasets[x], 
                batch_size=64, shuffle=True, num_workers=4)
                for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['test'].classes
print (dataset_sizes)
print (image_datasets)
print (class_names)

{'train': 50000, 'test': 10000}
{'train': Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./cifar10
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
           ), 'test': Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./cifar10
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
           )}
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


### 显示部分图片

In [ ]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     # mean = np.array([0.485, 0.456, 0.406])
#     # std = np.array([0.229, 0.224, 0.225])   # RGB
#     mean = np.array([0.1307])
#     std = np.array([0.3081])   # 灰度图
    
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated

# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['val']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

## 模型训练

In [5]:
#训练过程
def train_model(model, dataloaders, loss_function, optimizer, num_epochs=25):
  since = time.time()
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  train_loader = dataloaders['train']
  val_loader = dataloaders['test']

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1, num_epochs))
    print('-' * 10)
    # 记录把所有数据集训练+测试一遍需要多长时间
    startTick = time.time()

    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:  # 对于训练集的每一个batch
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      out = model(inputs)  # 送进网络进行输出
      _, preds = torch.max(out, 1)
      loss = loss_function(out, labels)  # 获得损失
  
      optimizer.zero_grad()  # 梯度归零
      loss.backward()  # 反向传播获得梯度，但是参数还没有更新
      optimizer.step()  # 更新梯度
      # scheduler.step()  # 更新学习率

      # statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
    
    epoch_loss = running_loss / dataset_sizes['train']
    epoch_acc = running_corrects.double() / dataset_sizes['train']
    print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                'train', epoch_loss, epoch_acc))

    model.eval()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in val_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
  
      out = model(inputs)  # 获得输出
  
      _, preds = torch.max(out, 1)
      loss = loss_function(out, labels)  # 获得损失
      # torch.max()返回两个结果，
      # 第一个是最大值，第二个是对应的索引值；
      # 第二个参数 0 代表按列取最大值并返回对应的行索引值，1 代表按行取最大值并返回对应的列索引值。
      # statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)  # 找出预测和真实值相同的数量，也就是以预测正确的数量
    
    epoch_loss = running_loss / dataset_sizes['test']
    epoch_acc = running_corrects.double() / dataset_sizes['test']
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('val', epoch_loss, epoch_acc))
                
    timeSpan = time.time() - startTick
    print("Time: %dS"%(timeSpan))
    print()

    if epoch_acc > best_acc:
      best_acc = epoch_acc
      best_model_wts = copy.deepcopy(model.state_dict())

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))
  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

## 搭建网络结构

In [6]:
# https://blog.csdn.net/qq_41683065/article/details/91368288
# https://www.zhihu.com/question/279878195
# 搭建网络
class myNet(nn.Module):
  def __init__(self):
    super(myNet,self).__init__()
    # 输入shape 3*32*32
    self.conv0 = torch.nn.Sequential(
          nn.Conv2d(3,64,3,padding=1),       # 64*32*32
          nn.Conv2d(64,64,3,padding=1),      # 64*32*32
          nn.MaxPool2d(2, 2),           # 64*16*16
          nn.BatchNorm2d(64),           # 64*16*16
          nn.ReLU()                # 64*16*16
    )

    self.conv1 = torch.nn.Sequential(
          nn.Conv2d(64,128,3,padding=1),      # 128*16*16
          nn.Conv2d(128, 128, 3,padding=1),   # 128*16*16
          nn.MaxPool2d(2, 2, padding=1),      # 128*9*9
          nn.BatchNorm2d(128),           # 128*9*9
          nn.ReLU()                # 128*9*9
    )
    
    self.conv2 = torch.nn.Sequential(
          nn.Conv2d(128,128, 3,padding=1),    # 128*9*9
          nn.Conv2d(128, 128, 3,padding=1),   # 128*9*9
          nn.Conv2d(128, 128, 1,padding=1),   # 128*11*11
          nn.MaxPool2d(2, 2, padding=1),    # 128*6*6
          nn.BatchNorm2d(128),          # 128*6*6
          nn.ReLU(),               # 128*6*6
    )
    
    self.conv3 = torch.nn.Sequential(
          nn.Conv2d(128, 256, 3,padding=1),   # 256*6*6
          nn.Conv2d(256, 256, 3, padding=1),  # 256*6*6
          nn.Conv2d(256, 256, 1, padding=1),  # 256*8*8
          nn.MaxPool2d(2, 2, padding=1),     # 256*5*5
          nn.BatchNorm2d(256),           # 256*5*5
          nn.ReLU()                # 256*5*5
    )

    self.conv4 = torch.nn.Sequential(
          nn.Conv2d(256, 512, 3, padding=1),  # 512*5*5
          nn.Conv2d(512, 512, 3, padding=1),  # 512*5*5
          nn.Conv2d(512, 512, 1, padding=1),  # 512*7*7
          nn.MaxPool2d(2, 2, padding=1),     # 512*4*4
          nn.BatchNorm2d(512),           # 512*4*4
          nn.ReLU()                # 512*4*4
    )

    self.out = torch.nn.Sequential(
          nn.Linear(512*4*4,1024),        # 1*1024
          nn.ReLU(),
          nn.Dropout2d(),             # 1*1024
          nn.Linear(1024,1024),          # 1*1024
          nn.ReLU(),
          nn.Dropout2d(),             # 1*1024
          nn.Linear(1024,10)           # 1*10
    )
      

  def forward(self,x):
    x = self.conv0(x)
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)

    x = x.view(-1,512*4*4)  # 展平

    x = self.out(x)

    return x


# 
model_ft=myNet()
model_ft.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

print(model_ft)


myNet(
  (conv0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
  )
  (conv1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    

## 训练

In [8]:
os.chdir("/content")
torch.save(model_ft,'draw.pth')
model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=20)


Epoch 1/20
----------
train Loss: 1.1954 Acc: 0.5678
val Loss: 1.1104 Acc: 0.6109
Time: 70S

Epoch 2/20
----------
train Loss: 1.1012 Acc: 0.6080
val Loss: 1.0464 Acc: 0.6413
Time: 70S

Epoch 3/20
----------
train Loss: 1.0205 Acc: 0.6411
val Loss: 1.0960 Acc: 0.6124
Time: 69S

Epoch 4/20
----------
train Loss: 0.9625 Acc: 0.6617
val Loss: 0.9686 Acc: 0.6697
Time: 69S

Epoch 5/20
----------
train Loss: 0.9078 Acc: 0.6795
val Loss: 0.9810 Acc: 0.6644
Time: 70S

Epoch 6/20
----------
train Loss: 0.8577 Acc: 0.7012
val Loss: 0.9282 Acc: 0.6714
Time: 69S

Epoch 7/20
----------
train Loss: 0.8178 Acc: 0.7180
val Loss: 1.0325 Acc: 0.6554
Time: 70S

Epoch 8/20
----------
train Loss: 0.7748 Acc: 0.7347
val Loss: 0.9834 Acc: 0.6786
Time: 70S

Epoch 9/20
----------
train Loss: 0.7354 Acc: 0.7460
val Loss: 0.8326 Acc: 0.7191
Time: 69S

Epoch 10/20
----------
train Loss: 0.6994 Acc: 0.7648
val Loss: 0.7900 Acc: 0.7395
Time: 69S

Epoch 11/20
----------
train Loss: 0.6686 Acc: 0.7719
val Loss: 0.762

### 保存

In [9]:
# 可视化工具
# https://lutzroeder.github.io/netron/
os.chdir("/content")
torch.save(model_ft,'draw.pth')

In [ ]:
# os.chdir("/content/drive/My Drive")
# !ls
# torch.save(model_ft,'light.pth')